In [20]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122542")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122542
Azure region: southcentralus
Subscription id: 8ca4d354-cab8-4c39-9d61-ea227d2daaad
Resource group: aml-quickstarts-122542


In [21]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException


cpu_cluster_name = "BankCC "

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [22]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import Dataset
import os

# Specify parameter sampler
ps = RandomParameterSampling(
	{
		'--C':uniform(.90, 1.10),
		'--max_iter':choice(80, 90, 100, 110, 120)
	}
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', compute_target='BankCC' , entry_script='train.py', pip_packages=[])

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling=ps, policy=policy, primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=20, max_concurrent_runs=4)

In [24]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [25]:
# Get your best run and save the model from that run.
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print("Best Accuracy from runs:", best_run_metrics["Accuracy"]  )

model = best_run.register_model(model_name = "hyperdriverun", model_path = "output/model.joblib")


Best Accuracy from runs: 0.910980320029428


ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path output/model.joblib in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_1bfff6041b527f249fc182243b0f1551ee9c2ff6a729dcc68d1a74bdf64bb002_d.txt', 'azureml-logs/65_job_prep-tvmps_1bfff6041b527f249fc182243b0f1551ee9c2ff6a729dcc68d1a74bdf64bb002_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_1bfff6041b527f249fc182243b0f1551ee9c2ff6a729dcc68d1a74bdf64bb002_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/112_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_0f79a58c-d30a-473c-a0b5-1ed08b7093d9.jsonl', 'logs/azureml/dataprep/python_span_l_0f79a58c-d30a-473c-a0b5-1ed08b7093d9.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']
                See https

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
file_url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dsauto = TabularDatasetFactory.from_delimited_files(path = file_url)

In [12]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dsauto)

ModuleNotFoundError: No module named 'train'

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="Classification",
    primary_metric="Accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=5)

In [ ]:
# Submit your automl run
from azureml.train.automl import Experiment
experiment = Experiment(ws, "automl_test_experiment")
run = experiment.submit(config = automl_config, show_output = True)

In [ ]:
# Retrieve and save your best automl model.
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()

In [ ]:
compute_target.delete()